In [ ]:
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

### Example
Let $U(\mathbf{z}) = \frac{1}{2}\left(\frac{\|\mathbf{z}\|-2}{0.4}\right)^2 - \log\left(e^{-0.5\left[\frac{\mathbf{z}_1 - 2}{0.6}\right]} + e^{-0.5\left[\frac{\mathbf{z}_1 + 2}{0.6}\right]}\right)$, and $p(\mathbf{z}) \propto e^{-U(\mathbf{z})}$ be the distribution we want to sample from.

Let's visualize the (unnormalized) density.

In [ ]:
def npdensity1(z):
    z = np.reshape(z, [z.shape[0], 2])
    z1, z2 = z[:, 0], z[:, 1]
    norm = np.sqrt(z1 ** 2 + z2 ** 2)
    exp1 = np.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = np.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - np.log(exp1 + exp2)
    return np.exp(-u)

def npdensity2(z):
    z = z.reshape(-1, 2)
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return np.exp(-u)

In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = npdensity1(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Density #1')
plt.show()

In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = npdensity2(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Density #2')
plt.show()

## Unadjusted Langevin Algorithm (ULA)

The Langevin SDE can be simulated using the Euler-Maruyama scheme as follows

$$X_{k+1} = X_{k} - \gamma_{k+1}\nabla U(X_k) + \sqrt{2\gamma_{k+1}}Z_{k+1}$$

where $\gamma_{k+1}$ is the step size and $Z_{k+1}$ is a sample from $\mathcal{N}(0, I)$ respectively at the $k+1$-th time-step.

In practice, there are various tricks to set $\gamma_{k+1}$ and $X_0$, the initial seed. However, in the following examples, I've used a constant step-size and have sampled $X_0$ from $\mathcal{N}(0, I)$.

The above simulation of Langevin dynamics can be used to draw samples from densities of the form $p(x) = \frac{e^{-U(x)}}{Z}$ where Z may or may not be known. It is assumed that the gradient of $U$ is $L$-Lipschtiz. 

In [ ]:
def potential1(z):
    z = z.view(-1, 2)
    z1, z2 = z[:, 0], z[:, 1]
    norm = torch.norm(z, p=2, dim=1)
    exp1 = torch.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = torch.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - torch.log(exp1 + exp2)
    return u

def potential2(z):
    z = z.view(-1, 2)
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return u

def torch_tunable_sigmoid(x,epsilon):
    return 1/(1+torch.exp(- epsilon * x))

def potential_ball(z,epsilon=5,r=2):
    norm=torch.norm(z,p=2,dim=1)
    p=torch.sigmoid((r-norm)*epsilon)
    return -torch.log(p)

def potential_square(z,epsilon=5,r=2):
    norm=torch.norm(z,p=torch.inf,dim=1)
    p=torch.sigmoid((r-norm)*epsilon)
    return -torch.log(p)

def potential_vote(z,epsilon=5,r=2):
    z = z.view(-1, 2)
    g_1=z[:,1]-z[:,0]
    g_2=z[:,1]+z[:,0]
    #g_3=2-z[:,1]
    S_1=torch.sigmoid(epsilon*g_1)+torch.sigmoid(epsilon*g_2)
    V_1=torch.sigmoid(epsilon*(S_1-0.5))
    vote_term=torch.sigmoid(epsilon*(V_1-0.5))

    norm=torch.norm(z,p=torch.inf,dim=1)
    init_term=torch.sigmoid((r-norm)*epsilon)

    full_term=vote_term*init_term

    return -torch.log(full_term)

In [ ]:
def unadjusted_langevin_algorithm(potential, n_samples=100000, step=0.1):
    burn_in = 10000
    Z0 = torch.randn(1, 2)
    Zi = Z0
    samples = []
    for i in tqdm(range(n_samples + burn_in)):
        Zi.requires_grad_()
        u = potential(Zi).mean()
        grad = torch.autograd.grad(u, Zi)[0]
        Zi = Zi.detach() - step * grad + np.sqrt(2 * step) * torch.randn(1, 2)
        samples.append(Zi.detach().numpy())
    return np.concatenate(samples, 0)[burn_in:]

def unadjusted_langevin_algorithm_gpu(potential, n_samples=100000, step=0.01, iters=10000):
    if not torch.cuda.is_available():
        print("No GPU")
        return
    
    device=torch.device('cuda')
    Z0 = torch.randn(n_samples, 2, device=device)
    Zi = Z0
    for i in tqdm(range(iters)):
        Zi.requires_grad_()
        u = potential(Zi).sum()
        grad = torch.autograd.grad(u, Zi)[0]
        #print(grad)
        Zi = Zi.detach() - step * grad + np.sqrt(2 * step) * torch.randn(n_samples, 2, device=device)
        #samples.append(Zi.detach().cpu().numpy())
    return Zi.detach().cpu().numpy()

Run the ULA and render the empirical density.

In [ ]:
samples1 = unadjusted_langevin_algorithm(potential1)
samples2 = unadjusted_langevin_algorithm(potential2)

In [ ]:
samples_gpu = unadjusted_langevin_algorithm_gpu(potential1,step=0.1)

In [ ]:
samples_gpu_2_list=[]
for i in range(10):
    samples_gpu_2_list.append(unadjusted_langevin_algorithm_gpu(potential2,n_samples=1000,step=0.01))

samples_gpu_2 = np.concatenate(samples_gpu_2_list)

In [ ]:
samples_gpu_ball = unadjusted_langevin_algorithm_gpu(lambda z: potential_ball(z,5,1.5),n_samples=100000,step=0.1,iters=20000)
samples_gpu_sq = unadjusted_langevin_algorithm_gpu(lambda z: potential_square(z,5,1.5),n_samples=100000,step=0.1,iters=20000)
samples_gpu_vote = unadjusted_langevin_algorithm_gpu(lambda z: potential_vote(z,5,2),n_samples=100000,step=0.001,iters=20000)

In [ ]:
samples_gpu_vote

In [ ]:
samples_gpu

In [ ]:
samples_gpu_2

In [ ]:
plt.hist2d(samples1[:,0], samples1[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Empirical Density #1')
plt.show()

In [ ]:
plt.hist2d(samples_gpu[:,0], samples_gpu[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Empirical Density #1 GPU')
plt.show()

In [ ]:
plt.hist2d(samples2[:,0], samples2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()

In [ ]:
plt.hist2d(samples_gpu_2[:,0], samples_gpu_2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()

In [ ]:
plt.hist2d(samples_gpu_ball[:,0], samples_gpu_ball[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Ball GPU')
plt.show()

In [ ]:
plt.hist2d(samples_gpu_sq[:,0], samples_gpu_sq[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Square GPU')
plt.show()

In [ ]:
plt.hist2d(samples_gpu_vote[:,0], samples_gpu_vote[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Square GPU')
plt.show()

## Metropolis-adjusted Langevin Algorithm (MALA)

New samples are proposed using the Euler-Maruyama scheme as before, but are accepted/rejected using the Metropolis-Hastings algorithm, i.e., the acceptance propability is given by

$$\alpha = \min \left\{1, \frac{p(x_{k+1})Q(x_k|x_{k+1})}{p(x_{k})Q(x_{k+1}|x_{k})}\right\}$$

where 

$$Q(x'|x) \propto \exp\left(-\frac{1}{4\gamma}\|x' - x + \gamma\nabla U(x)\|^2\right)$$

In [ ]:
def log_Q(potential, z_prime, z, step):
    z.requires_grad_()
    grad = torch.autograd.grad(potential(z).mean(), z)[0]
    return -(torch.norm(z_prime - z + step * grad, p=2, dim=1) ** 2) / (4 * step)

def log_Q_gpu(potential, z_prime, z, step):
    z.requires_grad_()
    grad = torch.autograd.grad(potential(z).sum(), z)[0]
    return -(torch.norm(z_prime - z + step * grad, p=2, dim=1) ** 2) / (4 * step)

def metropolis_adjusted_langevin_algorithm(potential, n_samples=100000, step=0.1):
    burn_in = 10000
    Z0 = torch.randn(1, 2)
    Zi = Z0
    samples = []
    pbar = tqdm(range(n_samples + burn_in))
    for i in pbar:
        Zi.requires_grad_()
        u = potential(Zi).mean()
        grad = torch.autograd.grad(u, Zi)[0]
        prop_Zi = Zi.detach() - step * grad + np.sqrt(2 * step) * torch.randn(1, 2)
        log_ratio = -potential(prop_Zi).mean() + potential(Zi).mean() +\
                    log_Q(potential, Zi, prop_Zi, step) - log_Q(potential, prop_Zi, Zi, step)
        if torch.rand(1) < torch.exp(log_ratio):
            Zi = prop_Zi
        samples.append(Zi.detach().numpy())
    return np.concatenate(samples, 0)[burn_in:]

def metropolis_adjusted_langevin_algorithm_gpu(potential, n_samples=100000, step=0.1, iters=10000):
    if not torch.cuda.is_available():
        print("No GPU")
        return
    
    torch.cuda.empty_cache()
    device=torch.device('cuda')
    Z0 = torch.randn(n_samples, 2, device=device)
    Zi = Z0
    for i in tqdm(range(iters)):
        #torch.cuda.empty_cache()
        Zi.requires_grad_()
        u = potential(Zi).sum()
        grad = torch.autograd.grad(u, Zi)[0]
        #print(grad)
        prop_Zi = Zi.detach() - step * grad + np.sqrt(2 * step) * torch.randn(n_samples, 2, device=device)
        log_ratio = -potential(prop_Zi) + potential(Zi) +\
                    log_Q_gpu(potential, Zi, prop_Zi, step) - log_Q_gpu(potential, prop_Zi, Zi, step)
        mask= torch.rand(n_samples,device=device) < torch.exp(log_ratio)
        #print(mask)
        mask=mask.double().view(n_samples,-1)
        Zi=mask*prop_Zi + (1-mask)*Zi.detach()
        #samples.append(Zi.detach().cpu().numpy())
    return Zi.detach().cpu().numpy()

In [ ]:
samples1 = metropolis_adjusted_langevin_algorithm(potential1)
samples2 = metropolis_adjusted_langevin_algorithm(potential2)

In [ ]:
sample1_adj_gpu = metropolis_adjusted_langevin_algorithm_gpu(potential1,n_samples=100000,iters=1000)

In [ ]:
sample1_adj_gpu

In [ ]:
sample2_adj_gpu = metropolis_adjusted_langevin_algorithm_gpu(potential2,n_samples=100000,iters=1000)

In [ ]:
sample2_adj_gpu

In [ ]:
samples_adj_gpu_ball = metropolis_adjusted_langevin_algorithm_gpu(lambda z: potential_ball(z,5,1.5),n_samples=100000,step=0.01,iters=1000)
samples_adj_gpu_sq = metropolis_adjusted_langevin_algorithm_gpu(lambda z: potential_square(z,5,1.5),n_samples=100000,step=0.01,iters=1000)
samples_adj_gpu_vote = metropolis_adjusted_langevin_algorithm_gpu(lambda z: potential_vote(z,5,2),n_samples=100000,step=0.01,iters=1000)

In [ ]:
samples_adj_gpu_vote

In [ ]:
plt.hist2d(samples1[:,0], samples1[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Empirical Density #1')
plt.show()

In [ ]:
plt.hist2d(sample1_adj_gpu[:,0], sample1_adj_gpu[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Empirical Density #1 GPU')
plt.show()

In [ ]:
plt.hist2d(samples2[:,0], samples2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()

In [ ]:
plt.hist2d(sample2_adj_gpu[:,0], sample2_adj_gpu[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()

In [ ]:
plt.hist2d(samples_adj_gpu_ball[:,0], samples_adj_gpu_ball[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Ball GPU')
plt.show()

In [ ]:
plt.hist2d(samples_adj_gpu_sq[:,0], samples_adj_gpu_sq[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Square GPU')
plt.show()

In [ ]:
plt.hist2d(samples_adj_gpu_vote[:,0], samples_adj_gpu_vote[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density Square GPU')
plt.show()

## Metropolis-Hastings Algorithm for comparison

In [ ]:
def metropolis_hastings(target_density, size=100000):
    burnin_size = 10000
    size += burnin_size
    x0 = np.array([[0, 0]])
    xt = x0
    samples = []
    for i in tqdm(range(size)):
        xt_candidate = np.array([np.random.multivariate_normal(xt[0], np.eye(2))])
        accept_prob = (target_density(xt_candidate))/(target_density(xt))
        if np.random.uniform(0, 1) < accept_prob:
            xt = xt_candidate
        samples.append(xt)
    samples = np.array(samples[burnin_size:])
    samples = np.reshape(samples, [samples.shape[0], 2])
    return samples

In [ ]:
samples1 = metropolis_hastings(npdensity1)
samples2 = metropolis_hastings(npdensity2)

In [ ]:
plt.hist2d(samples1[:,0], samples1[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Empirical Density #1')
plt.show()

In [ ]:
plt.hist2d(samples2[:,0], samples2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()